<a href="https://colab.research.google.com/github/arielle00/CMPE257-Fall23-Arielle-Shnayder/blob/main/Homework3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Somesh and I collaborated on this code

In [152]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import copy

dfTrain = np.loadtxt("ZipDigits.train",delimiter=" ",usecols=range(0, 257))

dfTest = np.loadtxt("ZipDigits.test",delimiter=" ",usecols=range(0, 257))

print("df train :",dfTrain.shape)
print("df test :", dfTest.shape)

filtered_dfTrain = []
filtered_dfTest = []

filtered_dfTrain = dfTrain[(dfTrain[:, 0] == 1) | (dfTrain[:, 0] == 5)]
filtered_dfTest = dfTest[(dfTest[:, 0] == 1) | (dfTest[:, 0] == 5)]

filtered_dfTrain[:, 0] = np.where(filtered_dfTrain[:, 0] == 1, 1, -1)
filtered_dfTest[:, 0] = np.where(filtered_dfTest[:, 0] == 1, 1, -1)

def intensity(image):
    pixels = image[1:]
    #print(pixels.shape)
    return np.mean(pixels)

def hor_symmetry(image):
    # 16 x 16 = 256
    image_2d = image[1:].reshape(16, 16)
    upper_half = image_2d[:8,:]
    lower_half = image_2d[8:,:]
    lower_half_flipped = np.flipud(lower_half)
    return np.mean(np.abs(upper_half - lower_half_flipped))

intensities = np.array([intensity(img) for img in filtered_dfTrain])
symmetries = np.array([hor_symmetry(img) for img in filtered_dfTrain])
labels = filtered_dfTrain[:, 0]

df_train = pd.DataFrame({'x0': 1, 'x1':intensities, 'x2':symmetries, 'y':labels})


intensities_test = np.array([intensity(img) for img in filtered_dfTest])
symmetries_test = np.array([hor_symmetry(img) for img in filtered_dfTest])
labels_test = filtered_dfTest[:, 0]
df_test = pd.DataFrame({'x0': 1, 'x1':intensities_test, 'x2':symmetries_test, 'y':labels_test})

df train : (7291, 257)
df test : (2007, 257)


,x0,x1,x2,y
0,1,-0.680133,0.123438,1.0
1,1,-0.350320,0.824453,-1.0
2,1,-0.497641,0.704906,-1.0
3,1,-0.750781,0.032672,1.0
4,1,-0.561766,0.109719,1.0
...,...,...,...,...
419,1,-0.108152,0.643930,-1.0
420,1,-0.645996,0.079367,1.0
421,1,-0.230039,0.696031,-1.0
422,1,-0.320828,0.641234,-1.0


In [153]:
data = pd.DataFrame(df_train.iloc[:,:3])
data_train = data
y = pd.DataFrame(df_train.iloc[:,3]).to_numpy()

In [154]:
num_hidden_units = 2
learning_rate = 0.5
num_layers = 2
epochs = int(2000000/len(data_train))
SEED = 40

In [155]:
def forward_propagation(x0, w_forward):
    s_forward = {}
    x_forward = {0: x0}

    for i in range(1, num_layers+1):
        s = np.dot(w_forward[i].T, x_forward[i-1])
        s_forward[i] = s

        activation = np.append(np.array([1]), np.tanh(s))
        x_forward[i] = activation.reshape(-1, 1)

    return x_forward, s_forward

In [156]:
def backward_propagation(x_forward, s_forward, w_forward, y, k):

    x_backward = copy.deepcopy(x_forward)
    w_backward = copy.deepcopy(w_forward)
    for i in range(len(x_backward)):
        x_backward[i] = x_backward[i][1:]

    for i in range(1,len(w_backward)+1):
        w_backward[i] = w_backward[i][1:]

    delta_backward = {}
    delta_backward[num_layers] = 2*(x_backward[num_layers]-y[k])*(1 - np.square(np.tanh(s_forward[num_layers])))

    for i in range(num_layers-1, 0, -1):
        tanh_derivative = 1-np.square(np.tanh(s_forward[i]))
        delta_backward[i] = np.multiply(tanh_derivative, (w_backward[i+1] * delta_backward[i+1]))

    return delta_backward

In [157]:
def gradientDescent(x_forward, delta_backward):
    gradients = {}
    for layer in range(1, num_layers+1):
        gradients[layer] = np.dot(x_forward[layer-1], delta_backward[layer].T)
    return gradients

In [158]:
def update_weights(w_forward, grad_des, learning_rate):
    for layer in range(1, num_layers+1):
        w_forward[layer] -= learning_rate * grad_des[layer]
    return w_forward

PART 3

In [ ]:
x = data.to_numpy()

np.random.seed(SEED)
w_forward = {}
w_forward[1] = np.random.randn(num_hidden_units, x.shape[1]).reshape(-1, 2)
w_forward[2] = np.random.randn(1, num_hidden_units+1).reshape(-1, 1)
in_sample_errors = []

for epoch in range(epochs):
    e_in = 0
    shuffled_indices = np.random.permutation(len(x))
    shuffled_x = x[shuffled_indices]
    shuffled_y = y[shuffled_indices]

    for i in range(len(shuffled_x)):
        x0 = shuffled_x[i, :].reshape(-1, 1)
        x_forward, s_forward = forward_propagation(x0, w_forward)
        delta_backward = backward_propagation(x_forward, s_forward, w_forward, shuffled_y, i)

        e_in += np.square(x_forward[num_layers][1:] - shuffled_y[i])

        gradients = gradientDescent(x_forward, delta_backward)
        w_forward = update_weights(w_forward, gradients, learning_rate)

    average_epoch_error = e_in / len(shuffled_x)
    in_sample_errors.append(e_in[0])


PART 4

In [ ]:
plt.title("Error Rate vs. Epochs")
plt.xlabel("Number of Epochs")
plt.ylabel("Error Rate (E_in)")
plt.plot(in_sample_errors)


In [ ]:

test_x = pd.DataFrame(df_test.iloc[:,:3]).to_numpy()
test_y = pd.DataFrame(df_test.iloc[:,3]).to_numpy()
np.random.seed(SEED)
correct_predictions = 0

for i in range(len(test_x)):
    x0 = test_x[i, :].reshape(-1, 1)
    x_forward, s_forward = forward_propagation(x0, w_forward)
    predicted_output = x_forward[num_layers][1][0]

    if(np.sign(predicted_output) == np.sign(test_y[i])):
        correct_predictions += 1

corr_class = correct_predictions/len(test_y)
incorrect = 1 - correct_predictions/len(test_y)
print("Correct classification: {:.3f}%".format(corr_class*100))
print("Test Error Rate: {:.3f}%".format(incorrect*100))

HP 2

In [ ]:
x = data.to_numpy()

np.random.seed(SEED)
w_forward = {}
w_forward[1] = np.random.randn(num_hidden_units, x.shape[1]).reshape(-1, 2)
w_forward[2] = np.random.randn(1, num_hidden_units+1).reshape(-1, 1)
in_sample_errors = []

for epoch in range(epochs):
    e_in = 0
    shuffled_indices = np.random.permutation(len(x))
    shuffled_x = x[shuffled_indices]
    shuffled_y = y[shuffled_indices]

    for i in range(32):
        x0 = shuffled_x[i, :].reshape(-1, 1)
        x_forward, s_forward = forward_propagation(x0, w_forward)
        delta_backward = backward_propagation(x_forward, s_forward, w_forward, shuffled_y, i)

        e_in += np.square(x_forward[num_layers][1:] - shuffled_y[i])

        gradients = gradientDescent(x_forward, delta_backward)
        w_forward = update_weights(w_forward, gradients, learning_rate)

    average_epoch_error = e_in / len(shuffled_x)
    in_sample_errors.append(e_in[0])

In [ ]:
plt.title("Error Rate vs. Epochs for Mini Batch")
plt.xlabel("Number of Epochs")
plt.ylabel("Error Rate (E_in)")
plt.plot(in_sample_errors)

In [ ]:

test_x = pd.DataFrame(df_test.iloc[:,:3]).to_numpy()
test_y = pd.DataFrame(df_test.iloc[:,3]).to_numpy()
np.random.seed(SEED)
correct_predictions = 0

for i in range(len(test_x)):
    x0 = test_x[i, :].reshape(-1, 1)
    x_forward, s_forward = forward_propagation(x0, w_forward)
    predicted_output = x_forward[num_layers][1][0]

    if(np.sign(predicted_output) == np.sign(test_y[i])):
        correct_predictions += 1

corr_class = correct_predictions/len(test_y)
incorrect = 1 - correct_predictions/len(test_y)
print("Correct classification: {:.3f}%".format(corr_class*100))
print("Test Error Rate: {:.3f}%".format(incorrect*100))